In [1]:
import torch
import librosa
import soundfile as sf
import skimage.measure
import numpy as np


d:\ProgramData\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
from torch import nn
import torch.nn as nn
import torch
import torch.nn.functional as F

In [39]:
import soundfile as sf
import tqdm

In [3]:
class CnnAudioNet(nn.Module):
    def __init__(self,numClasses):
        super(CnnAudioNet,self).__init__()
        self.num_classes = numClasses
        self.fc_features = 64
        self.C1 = nn.Conv2d(2,32,3,padding=1)
        self.C11 = nn.Conv2d(32,32,3,padding=1)
        self.C2 = nn.Conv2d(32,64,3,padding=1)
        self.C22 = nn.Conv2d(64,64,3,padding=1)
        self.C3 = nn.Conv2d(64,128,3,padding=1)
        self.C33 = nn.Conv2d(128,128,3,padding=1)
        
        self.BN1 = nn.BatchNorm2d(32)
        self.BN11 = nn.BatchNorm2d(32)
        self.BN2 = nn.BatchNorm2d(64)
        self.BN22 = nn.BatchNorm2d(64)
        self.BN3 = nn.BatchNorm2d(128)
        self.BN33 = nn.BatchNorm2d(128)

        self.BNFC = nn.BatchNorm1d(self.fc_features)
        self.maxpool1 = nn.MaxPool2d(2,2)
        self.maxpool2 = nn.MaxPool2d((2,2),(2,2))
        self.softmax = nn.Softmax(dim=1)
        
        self.flatten = torch.nn.Flatten()
        
        # self.fc1 = nn.Linear(64*32*32,self.fc_features)
        # self.fc2 = nn.Linear(self.fc_features + 1,self.num_classes) 
        self.fc1 = nn.Linear(1152,self.fc_features)
        self.fc2 = nn.Linear(self.fc_features, 1) 
        self.dropout = nn.Dropout(0.35)

        self.BNout = nn.BatchNorm1d(1)
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
  
    def forward(self,x,timestamp):
        # add sequence of convolutional and max pooling layers
        x = self.maxpool1(F.relu(self.BN1(self.C1(x))))
        x = self.maxpool1(F.relu(self.BN11(self.C11(x))))
        x = self.maxpool1(F.relu(self.BN2(self.C2(x))))
        x = self.maxpool1(F.relu(self.BN22(self.C22(x))))
        x = self.maxpool1(F.relu(self.BN3(self.C3(x))))
        x = self.maxpool1(F.relu(self.BN33(self.C33(x))))
        # flatten image input
        x = x.view(-1,1152)
        x =  self.BNFC(self.fc1(x))
        x = self.dropout(x)
        if not time_test:
          x = torch.cat((x, extra_feature/240), 1) #normalize extra_feature
          x = self.fc2(x)
        else:
          # x = torch.cat((x, (extra_feature-2)/100), 1) #normalize extra_feature
          x = self.fc2(x)
          x = self.BNout(x)
          x = self.sigmoid(x) * 1440
        #x = self.softmax(x) #already included in CrossEntropyLoss
        return x
        

class HPSSCnnAudioNet(nn.Module):
    def __init__(self,numClasses):
        super(HPSSCnnAudioNet,self).__init__()
        self.num_classes = numClasses
        self.fc_features = 64
        self.C1 = nn.Conv2d(2,32,3,padding=1)
        self.C11 = nn.Conv2d(32,32,3,padding=1)
        self.C2 = nn.Conv2d(32,64,3,padding=1)
        self.C22 = nn.Conv2d(64,64,3,padding=1)
        self.C3 = nn.Conv2d(64,128,3,padding=1)
        self.C33 = nn.Conv2d(128,128,3,padding=1)
        
        self.BN1 = nn.BatchNorm2d(32)
        self.BN11 = nn.BatchNorm2d(32)
        self.BN2 = nn.BatchNorm2d(64)
        self.BN22 = nn.BatchNorm2d(64)
        self.BN3 = nn.BatchNorm2d(128)
        self.BN33 = nn.BatchNorm2d(128)

        self.BNFC = nn.BatchNorm1d(self.fc_features)
        self.maxpool1 = nn.MaxPool2d(2,2)
        self.maxpool2 = nn.MaxPool2d((2,2),(2,2))
        self.softmax = nn.Softmax(dim=1)
        
        self.flatten = torch.nn.Flatten()
        
        # self.fc1 = nn.Linear(64*32*32,self.fc_features)
        # self.fc2 = nn.Linear(self.fc_features + 1,self.num_classes) 
        self.fc1 = nn.Linear(1152,self.fc_features)
        self.fc2 = nn.Linear(self.fc_features, 1) 
        self.dropout = nn.Dropout(0.35)

        self.BNout = nn.BatchNorm1d(1)
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
  
    def forward(self,x,timestamp):
        # add sequence of convolutional and max pooling layers
        x = self.maxpool1(F.relu(self.BN1(self.C1(x))))
        x = self.maxpool1(F.relu(self.BN11(self.C11(x))))
        x = self.maxpool1(F.relu(self.BN2(self.C2(x))))
        x = self.maxpool1(F.relu(self.BN22(self.C22(x))))
        x = self.maxpool1(F.relu(self.BN3(self.C3(x))))
        x = self.maxpool1(F.relu(self.BN33(self.C33(x))))
        # flatten image input
        x = x.view(-1,1152)
        x =  self.BNFC(self.fc1(x))
        x = self.dropout(x)
        # if not time_test:
        #   x = torch.cat((x, extra_feature/240), 1) #normalize extra_feature
        #   x = self.fc2(x)
        # else:
          # x = torch.cat((x, (extra_feature-2)/100), 1) #normalize extra_feature
        x = self.fc2(x)
        x = self.BNout(x)
        x = self.sigmoid(x) * 1440
        #x = self.softmax(x) #already included in CrossEntropyLoss
        return x
        

In [4]:
hpss_model = HPSSCnnAudioNet(1)
mono_model = CnnAudioNet(1)

In [5]:
# mono_model.load_state_dict(torch.load(r"D:\git\Acoustic-Scene-Classification-and-Time-of-Day-Estimation\weights\Mono\spot_2_3_4_5_6_7_8_9_10_11\MonoCnnAudioNet_74.pt"))
hpss_model.load_state_dict(torch.load(r"D:\git\Acoustic-Scene-Classification-and-Time-of-Day-Estimation\weights\HPSS\spot_2_3_4_5_6_7_8_9_10_11\HPSSCnnAudioNet_30.pt"))

<All keys matched successfully>

In [6]:


import torch.nn as nn
import torch.nn.functional as F

# define the NN architecture
class ConvAutoencoder(nn.Module):
    def __init__(self):
        super(ConvAutoencoder, self).__init__()
        # conv layer (depth from 1 --> 16), 3x3 kernels
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)  
        # conv layer (depth from 16 --> 2), 3x3 kernels
        self.conv2 = nn.Conv2d(16, 3, 3, padding=1)
        # pooling layer to reduce x-y dims by two; kernel and stride of 2
        self.pool1 = nn.MaxPool2d(2,stride=(2,2),padding=1)
        self.pool2 = nn.MaxPool2d(2,stride=(2,2),padding=0)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        # add second hidden layer
        x = F.relu(self.conv2(x))
        x = self.pool2(x)  # compressed representation
                
        return x

In [26]:
def get_sig(filename, sample_rate=16000):
    y, sr = librosa.load(filename,sr=sample_rate, offset=0, duration=30)
    return y
    
def createMonoSpectrogram(filename, n_mels=128, n_fft=2048, hop_length=512):
  sig, sr = sf.read(filename)
  sig = sig[:,0]
  S = librosa.feature.melspectrogram(y=sig, sr=sr, hop_length=hop_length, n_fft=n_fft, n_mels=n_mels)
  return S

def createMonoSpectrogram(sig, sr, n_mels=128, n_fft=2048, hop_length=512):
  sig = sig[:,0]

  S = librosa.feature.melspectrogram(y=sig, sr=sr, hop_length=hop_length, n_fft=n_fft, n_mels=n_mels)
  return S

def createHPSS(y, n_fft=2544, hop_length=520):

  D = librosa.stft(y,n_fft=2544,hop_length=520)
  D_harmonic, D_percussive = librosa.decompose.hpss(D, margin=10)
  D_harmonic_downsampled = skimage.measure.block_reduce(D_harmonic, (5,10), func=np.max, cval=0)
  D_percussive_downsampled = skimage.measure.block_reduce(D_percussive, (5,10), func=np.max, cval=0)
  S = np.array([D_harmonic_downsampled,D_percussive_downsampled])
  return S


class Dataset(torch.utils.data.Dataset):
    def __init__(self, df, sample_rate=16000):
        'Initialization'

        self.df = df
        self.sample_rate = sample_rate
        
    def __len__(self):
        'Denotes the total number of samples'
        return len(self.df)

    def __getitem__(self, index):
        try:
          'Generates one sample of data'
          file_name = self.df.path[index]
          sig = get_sig(file_name, sample_rate)
          # S = createHPSS(sig)
          Im_hpss = None#torch.from_numpy(S).float()

          return sig, Im_hpss, self.df.time_of_the_day[index]
        except:
           return None,None,None

In [27]:
import pandas as pd

In [28]:
test_df = pd.read_csv(r"D:\git\Acoustic-Scene-Classification-and-Time-of-Day-Estimation\TestSet\labels.csv")

In [29]:
train_df = pd.read_csv(r"D:\git\Acoustic-Scene-Classification-and-Time-of-Day-Estimation\DataSet\labels.csv")

In [30]:
train_df['name'] = train_df['filename'].apply(lambda x:x.split('.')[0])
test_df['name'] = test_df['filename'].apply(lambda x:x.split('.')[0])

In [31]:
from pathlib import Path

files = [str(x) for x in Path(r'D:\Birds_audio').rglob('*.flac')]

df_lst = [(x,x.split('\\')[-1].split('_')[0],x.split('\\')[-1].split('.')[0]) for x in files]

files_df = pd.DataFrame(df_lst,columns=['path','spot','name'])

In [32]:
files_df.spot = files_df.spot.astype('int64')

In [33]:
merged_train = train_df.merge(files_df,'inner',left_on=['name','label'],right_on=['name','spot'])

In [34]:
merged_test = test_df.merge(files_df,'inner',left_on=['name','label'],right_on=['name','spot'])

In [35]:
def filename_to_time(file_name) :  
    timestam_str = file_name.split("_", 1)[1]
    timestam_str = (timestam_str.split(".", 1)[0]).split(" ",1)[0]
    timestamp = int(timestam_str)/1000 # quantization of 30-minutes
    return timestamp

In [36]:
files_df['time_of_the_day'] = files_df.name.apply(lambda x: filename_to_time(x)) 

In [37]:
s = set(merged_train.name.values )
files_df['in_train'] = files_df['name'].apply(lambda x: x in s )

In [38]:
s = set(merged_test.name.values )
files_df['in_test'] = files_df['name'].apply(lambda x: x in s )

In [20]:
ds = Dataset(files_df)

In [25]:
# sig = get_sig(files_df.path[0])
# S = createHPSS(sig)

In [24]:
hpss_model = hpss_model.eval().cuda()

In [26]:
def helper(index):

    try:
        file_name = files_df.path[index]
        sig = get_sig(file_name)
        S = createHPSS(sig)
        Im_hpss = torch.from_numpy(S).float()

        torch.save(Im_hpss, fr'G:\My Drive\birds_audio\{files_df.spot[index]}_{files_df.name[index]}.hpss')
        # pred_times.append(hpss_model(hpss.cuda().unsqueeze(0), timestamp=None).item())
        dy = librosa.resample(sig,orig_sr=44100,target_sr=16000).astype('float32')
        sf.write(fr'G:\My Drive\birds_audio\{files_df.spot[index]}_{files_df.name[index]}.flac',dy,16000)
    except:
        import traceback
        print(traceback.format_exc())


In [27]:
if __name__ == '__main__':
    # dl = torch.utils.data.DataLoader(ds,batch_size=1,num_workers=3,prefetch_factor=4)
    pred_times = []
    with torch.no_grad():
        for i,(y,hpss, timestamp) in enumerate(tqdm.tqdm_notebook(ds,total=len(ds))):
            if y is None:
                continue
            try:
                torch.save(hpss, files_df.path[i]+'.hpss')
                pred_times.append(hpss_model(hpss.cuda().unsqueeze(0), timestamp=None).item())
                # dy = librosa.resample(y,orig_sr=44100,target_sr=16000).astype('float32')
                # sf.write(fr'G:\My Drive\birds_audio\{files_df.spot[i]}_{files_df.name[i]}.flac',dy,16000)
            except:
                import traceback
                print(traceback.format_exc())

C:\Users\danny\AppData\Local\Temp\ipykernel_4996\1921238296.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i,(y,hpss, timestamp) in enumerate(tqdm.tqdm_notebook(ds,total=len(ds))):


  0%|          | 0/87245 [00:00<?, ?it/s]

C:\Users\danny\AppData\Local\Temp\ipykernel_4996\2442042022.py:43: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at ..\aten\src\ATen\native\Copy.cpp:276.)
  Im_hpss = torch.from_numpy(S).float()


In [94]:
res = hpss_model(torch.tensor(hpss).unsqueeze(0), timestamp=None)

C:\Users\danny\AppData\Local\Temp\ipykernel_11524\3782375586.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res = hpss_model(torch.tensor(hpss).unsqueeze(0), timestamp=None)


In [95]:
res

tensor([[316.8122]], grad_fn=<MulBackward0>)

In [32]:
test_files = df.filename.apply(lambda x: x.split('.')[0])

In [98]:
'20200401_142800' in test_files

False

In [126]:
test_files[0] == '20200330_031600'

True

In [131]:
test_files.values

array(['20200330_031600', '20200330_051200', '20200330_054000', ...,
       '20200407_230400', '20200408_021200', '20200408_035200'],
      dtype=object)

In [132]:
'20200330_031600' in test_files.values

True

In [115]:
test_files[0]

'20200330_031600'

In [133]:
files_df['in_test'] = files_df['name'].apply(lambda x: x in test_files.values)

In [134]:
files_df[files_df.in_test==True]

,path,spot,name,in_test
22,F:\bird identification2\Quarantine recordings ...,1,20200430_040000,True
24,F:\bird identification2\Quarantine recordings ...,1,20200430_070000,True
30,F:\bird identification2\Quarantine recordings ...,1,20200430_150000,True
34,F:\bird identification2\Quarantine recordings ...,1,20200430_200000,True
53,F:\bird identification2\Quarantine recordings ...,1,20200501_220000,True
...,...,...,...,...
108667,F:\bird identification2\Quarantine recordings ...,19,20200408_171200,True
108693,F:\bird identification2\Quarantine recordings ...,19,20200408_195400,True
108722,F:\bird identification2\Quarantine recordings ...,19,20200409_072600,True
108735,F:\bird identification2\Quarantine recordings ...,19,20200409_115200,True


In [136]:
files_df

,path,spot,name,in_test
0,F:\bird identification2\Quarantine recordings ...,1,20200428_230000,False
1,F:\bird identification2\Quarantine recordings ...,1,20200429_000000,False
2,F:\bird identification2\Quarantine recordings ...,1,20200429_030000,False
3,F:\bird identification2\Quarantine recordings ...,1,20200429_040000,False
4,F:\bird identification2\Quarantine recordings ...,1,20200429_050000,False
...,...,...,...,...
112153,F:\bird identification2\Quarantine recordings ...,9,20200329_161400,False
112154,F:\bird identification2\Quarantine recordings ...,9,20200329_162300,False
112155,F:\bird identification2\Quarantine recordings ...,9,20200329_163900,False
112156,F:\bird identification2\Quarantine recordings ...,9,20200329_165000,False


In [37]:
files_df

,path,spot,name,in_test
0,F:\bird identification2\Quarantine recordings ...,1,20200428_230000,False
1,F:\bird identification2\Quarantine recordings ...,1,20200429_000000,False
2,F:\bird identification2\Quarantine recordings ...,1,20200429_030000,False
3,F:\bird identification2\Quarantine recordings ...,1,20200429_040000,False
4,F:\bird identification2\Quarantine recordings ...,1,20200429_050000,False
...,...,...,...,...
112153,F:\bird identification2\Quarantine recordings ...,9,20200329_161400,False
112154,F:\bird identification2\Quarantine recordings ...,9,20200329_162300,False
112155,F:\bird identification2\Quarantine recordings ...,9,20200329_163900,False
112156,F:\bird identification2\Quarantine recordings ...,9,20200329_165000,False
